# Demo 1: Using Data from a SMART to Address Main Effects Comparisons

</br>

This material has been developed for [Getting SMART About Adaptive Interventions in Education](https://d3lab.isr.umich.edu/training/) led by [d3lab](https://d3lab.isr.umich.edu). 
    
Notebooks were developed by [Nicholas J. Seewald](https://nickseewald.com). 
SAS code originally written by Daniel Almirall, Inbal Nahum-Shani, and Susan A. Murphy.
The code was translated into R by Audrey Boruvka and Nicholas J. Seewald.


**Disclaimer:** This file provides example R code to analyze *simulated* data that was generated to mimic data arising from the ADHD SMART study (PI: William Pelham). An accompanying handout ("ADHD Handout.pdf") describes the variables in the data.

### Contents:
- [Import and Summarize Data](#data)
- [Part (a): Investigating the main effect of first-stage intervention options](#part-a)
- [Part (b): Main effect of second-stage options / tactics](#part-b)
- [Part (c): Sample Size for Primary Aims involving Main Effect Comparisons](#part-c)

<hr>

## Background
We'll be using the ADHD SMART (PI: Pelham) for this tutorial. A diagram of the study is below.

<img src="assets/adhd-smart-diagram.jpg" alt="ADHD SMART diagram" style="width: 50%;"/>

## <a name="setup"></a> Setup
We'll start by loading some packages, which have already been installed into the notebook environment.

In [ ]:
if (!length(find.package("geepack", quiet = TRUE))) 
    install.packages("geepack", dependencies = TRUE)

The file `functions.R` in this project contains code for a function called `estimate()` that will help us produce cleaner output from some of the models we'll fit in this module. Advanced R users are encouraged to look at this file to see how these functions work; otherwise, just know that this code will help us mimic SAS's estimate statements which are used in the training slides.

In [1]:
source('functions.R')

function 'estimate' loaded successfully.


## <a name="data"></a> Import and Summarize Data

Let's read in the dataset `adhd-simulated-dataset.csv`. This is a .csv file, so we'll import it into R using the `read.csv` function. We'll call the dataset `adhd`. This will be the name we use for the dataset throughout the remainder of the code. *Remember that this is **simulated** data and does not necessarily reflect actual findings of the ADHD SMART study.*

We will also rename all of the variables to be lowercase, since R is case-sensitive. Lowercase variable names are easier to type and remember!

In [2]:
adhd <- read.csv("assets/adhd-simulated-dataset.csv")

# R is case-sensitive! Avoid issues by changing variable names to all lowercase
names(adhd) <- tolower(names(adhd))

# Sort data by ID
adhd <- adhd[order(adhd$id), ]

Now that we've read in the data, we can view the first handful of rows using the `head` function.

In [3]:
head(adhd)

id o11 o12        o13 o14 a1 r o21 o22 a2 y rerand
1 1  0    1.1388268 0   1   -1 0  2  1   -1 3 1     
2 2  0    0.1974932 0   1   -1 0  6  1   -1 3 1     
3 3  1    0.3507961 0   0   -1 0  2  0    1 1 1     
4 4  1   -0.4076125 0   1   -1 0  6  1    1 4 1     
5 5  0   -0.7040368 1   0   -1 1 NA  0   NA 4 0     
6 6  1   -1.3398180 0   1   -1 0  1  0   -1 5 1

We can see that `adhd` contains a number of variables. We summarize them below. A more detailed summary is available in [this handout](https://gettingsmartworkshop-nseewald.notebooks.azure.com/j/files/ADHD_Data_Description_Handout.pdf).

| Variable Name | Variable Type | Description 
| -- | -- | :---
| `id` | Integer | Student ID number 
| `o11` | Baseline covariate; binary (0/1) | Oppositional defiant disorder diagnosis (1 = yes, 0 = no) 
| `o12` | Baseline covariate; continuous | ADHD severity at end of previous school year (0 - 3; higher values are fewer symptoms) 
| `o13` | Baseline covariate; binary (0/1) | Medication prior to first-stage intervention (1 = yes; 0 = no) 
| `o14` | Baseline covariate; binary (0/1) | Race (1 = white; 0 = non-white) 
| `a1` | First-stage intervention indicator (-1/1) | -1 = MED; -1 = BMOD 
| `r` | Response indicator (0/1) | 0 = Participant did not respond to first-stage intervention;  1 = participant responded
| `o21` | Intermediate outcome; continuous | Number of months until non-response (max of 8 months) 
| `o22` | Intermediate outcome; binary (0/1) | Adherence to first-stage intervention (1 = high adherence; 0 = otherwise)
| `a2` | Second-stage intervention indicator (-1/1) | -1 = AUGMENT; 1 = INTENSIFY
| `y` | Outcome (continuous) | Participant's school performance at end of school year (1-5; higher is better)

Whenever we start working with a dataset, it's helpful to summarize the variables to get a sense of what we're working with.

In [4]:
## Brief summary statistics
summary(adhd)

       id              o11              o12                o13        
 Min.   :  1.00   Min.   :0.0000   Min.   :-2.85949   Min.   :0.0000  
 1st Qu.: 38.25   1st Qu.:0.0000   1st Qu.:-0.85900   1st Qu.:0.0000  
 Median : 75.50   Median :0.0000   Median :-0.04929   Median :0.0000  
 Mean   : 75.50   Mean   :0.3533   Mean   :-0.12059   Mean   :0.3133  
 3rd Qu.:112.75   3rd Qu.:1.0000   3rd Qu.: 0.55809   3rd Qu.:1.0000  
 Max.   :150.00   Max.   :1.0000   Max.   : 2.43969   Max.   :1.0000  
                                                                      
      o14               a1           r             o21             o22        
 Min.   :0.0000   Min.   :-1   Min.   :0.00   Min.   :1.000   Min.   :0.0000  
 1st Qu.:1.0000   1st Qu.:-1   1st Qu.:0.00   1st Qu.:3.000   1st Qu.:0.0000  
 Median :1.0000   Median : 0   Median :0.00   Median :5.000   Median :0.0000  
 Mean   :0.8067   Mean   : 0   Mean   :0.34   Mean   :4.586   Mean   :0.4533  
 3rd Qu.:1.0000   3rd Qu.: 1   3rd Qu

We can also get the standard deviation for each of the variables. We do this by using `apply()`, which applies a function to the rows (1) or columns (2) of a `data.frame`. Here, we `apply` `sd` to the columns of `adhd`:

In [5]:
round(apply(adhd, 2, sd, na.rm = T), 3)

id    o11    o12    o13    o14     a1      r    o21    o22     a2      y 
43.445  0.480  1.014  0.465  0.396  1.003  0.475  2.015  0.499  1.005  1.281 
rerand 
 0.475

In [6]:
## Frequency table of the initial treatment, early response by week 8, and second
## stage treatments
table(adhd$a1)
table(adhd$r)
with(adhd, table(a2, r, useNA = "ifany"))  # cross-tabulate a2 and


-1  1 
75 75 


 0  1 
99 51 

      r
a2      0  1
  -1   49  0
  1    50  0
  <NA>  0 51

Let's create some frequency tables to see how our randomization went (how many children were assigned to each intervention option?) and to investigate the proportion of responders.

In [7]:
table(adhd$a1)


-1  1 
75 75 

In [8]:
table(adhd$r)


 0  1 
99 51 

In [9]:
table(adhd$a2, adhd$r, useNA = "ifany")

      
        0  1
  -1   49  0
  1    50  0
  <NA>  0 51

In [10]:
adhd$o11c <- with(adhd, o11 - mean(o11))
adhd$o12c <- with(adhd, o12 - mean(o12))
adhd$o13c <- with(adhd, o13 - mean(o13))
adhd$o14c <- with(adhd, o14 - mean(o14))
## o21 is measured among non-responders only
adhd$o22c <- with(adhd, o22 - mean(o22))

## center baseline variables using mean among non-responders
adhd$o11cnr <- adhd$o12cnr <- adhd$o13cnr <- adhd$o14cnr <- NA
adhd$o21cnr <- adhd$o22cnr <- NA
adhd$o11cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o11 - mean(o11))
adhd$o12cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o12 - mean(o12))
adhd$o13cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o13 - mean(o13))
adhd$o14cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o14 - mean(o14))
adhd$o21cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o21 - mean(o21))
adhd$o22cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o22 - mean(o22))

## <a name="part-a"></a> Part (a): Main Effect of First-Stage Intervention Options

Let's perform a regression analysis to compare first-line treatments on the end of study outcome: school performance (`Y`). 

To start, we'll load the package `geepack`, which contains the function `geeglm`. We'll use `geeglm` for the regression. This is analogous to `PROC GENMOD` in SAS.

In [11]:
library(geepack)

Now we fit the model. Model syntax in R works like this: 

`DependentVariable ~ covariate + covariate + ... + covariate`

We also need to specify an `id` variable (`geeglm` is built for repeated-measures analysis) and the `data` set we're working with. We pass these arguments to `geeglm` by using `argument = value` syntax.

Let's see what the code looks like:

In [12]:
model1 <- geeglm(y ~ a1 + o11c + o12c + o13c + o14c,
                 id = id, data = adhd)
round(coef(model1), 3)

(Intercept)          a1        o11c        o12c        o13c        o14c 
      2.953       0.093      -0.275      -0.490       0.052       0.529

In `model1`, we've fit the regression

$$ E[Y\mid A_1, \mathbf{O}_1] = b_0 + b_1 A_1 + b_2 O_{11c} + b_3 O_{12c} + b_4 O_{13c} + b_5 O_{14c}.$$

`functions.R`, which we imported earlier, contains a function called `estimate` which we can use to mimic the estimate statements in `PROC GENMOD`. It takes two arguments: the first is the model we fit above, and the second is a matrix which specifies "contrasts" of interest. This is analogous to the `PROC GENMOD` syntax *except* the arguments are not identified, and are instead "positional". The first element is the intercept, the second is $A_1$, etc.

In [13]:
estimate(model1,
         rbind("Mean Y under BMOD"   = c(1,  1, 0, 0, 0, 0),
               "Mean Y under MED"    = c(1, -1, 0, 0, 0, 0),
               "Between groups diff" = c(0,  2, 0, 0, 0, 0)))

                    Estimate 95% LCL 95% UCL     SE p-value
Mean Y under BMOD     3.0459  2.7859  3.3059 0.1326  <1e-04
Mean Y under MED      2.8608  2.6008  3.1208 0.1326  <1e-04
Between groups diff   0.1851 -0.1849  0.5551 0.1888  0.3269

Let's talk about where the Estimate and p-value columns in the above output are coming from. *Remember that the above results are using simulated data and do not necessarily reflect actual results of the ADHD SMART study.*

Because we've centered the baseline covariates $O_{1.c}$, we can estimate the mean school performance $Y$ under BMOD and at mean levels of all baseline covariates with $\hat{b}_0 + \hat{b}_1$, and under MED at mean levels of all baseline covariates with $\hat{b}_0 - \hat{b}_1$. Our estimate of the mean between-groups difference is thus $(\hat{b}_0 + \hat{b}_1)-(\hat{b}_0 - \hat{b}_1) = 2\hat{b}_1$. We estimate, then, that mean school performance among children whose sequences of treatment began with BMOD is slightly higher than mean school performance of children who started on MED; however, this difference is not statistically significant (see below).

The p-value column corresponds to the test of the null hypothesis $H_0: \mathbf{c}^\top \mathbf{b} = 0$ against the alternative $H_1: \mathbf{c}^\top \mathbf{b} \neq 0$, where $\mathbf{c}$ is the "contrast vector" that corresponding to the appropirate combination of $b$'s (see the corresponding row in the use of `estimate` above). For example, the p-value of 0.3269 in the last row of the table corresponds to the test $H_0: 2b_1=0$ against the alternative $H_1 \neq 0$. At the 5% significance level, we fail to reject $H_0$: there is insufficient evidence to suggest that there is a difference in mean school performance between children whose sequence of treatments began with MED and those whose sequences began with BMOD.


### Side Analysis: Effect of Stage 1 Options on Non-Response Rate

We can use data from a SMART to examine the impact of the first-stage treatments on non-response rate: perhaps one intervention option yields more responders than others. This analysis is *not particularly useful* in the context of developing an adaptive intervention because short-term response does not incorporate the effects of future treatment decisions.  Nonetheless, is still interesting in its own right, and we analyze it here for completeness.

For simplicity, we use a simple frequency table with a chi-squared test (of indpedendence) to compare the proportions responding to each initial treatment. In practice, we can also use logistic regression to adjust for baseline covariates (which is likely preferable). 

First, we'll construct a 2x2 table of `a1` and `r`. Here, we use the function `with()` to avoid some typing. `with()` lets you "attach" a `data.frame` "locally", meaning you don't have to type `adhd$` in front of every variable name. `with()` takes two arguments: the first is a `data.frame` you want to locally attach; the second is the code you want to run with that `data.frame` attached. Let's check it out:

In [14]:
with(adhd, table(a1, r))

    r
a1    0  1
  -1 47 28
  1  52 23

We've now cross-tabulated `a1` and `r`. We can see that 28 children who received MED in the first stage responded; 23 children who received BMOD responded. We can now perform a chi-squared test of independence to see if there is a statistically-significant difference in response rates. *(Note: the `correct = F` argument turns off the so-called Yates continuity correction for the chi-squared test in order to match the output of SAS)*

In [15]:
with(adhd, chisq.test(table(a1, r), correct = F))


	Pearson's Chi-squared test

data:  table(a1, r)
X-squared = 0.74272, df = 1, p-value = 0.3888


In terms of early response rate, initial MED is slighly better (vs. BMOD) by 7%, but there is not a statistically significant difference in response rates between the first stage options ($p$=0.39). 

## <a name="part-b"></a>Part (b): Main effect of second-stage options among non-responders

Let's now investigate the second common aim of a SMART: comparing the effects of second-stage intervention options among non-responders. Here, we use the term *tactics* to describe these options, as we are comparing *INTENSIFY*ing the original treatment versus *AUGMENT*ing it with the other option. While the *INTENSIFY* intervention is different depending on the first-stage intervention (intensified *BMOD* is not the same as intensified *MED*), the strategy is the same: we're intensifying the original intervention.

As before, we'll fit a model for the main effect of the second stage treatment (`a2`), adjusted for covariates. This time, the covariates we adjust for are centered at the mean *among non-responders*, as these individuals are the only ones contributing to our analysis. We're also allowed to adjust for covariates collected in the second stage ($O_{2.}$), since they came *before* both our treatment indicator `a2` and our outcome `y`.

In [16]:
model2 <- geeglm(y ~ a2 + o11cnr + o12cnr + o13cnr + o14cnr + o21cnr + o22cnr,
                 id = id,
                 data = adhd, 
                 # Only use non-responders
                 subset = r == 0)

estimate(model2,
         rbind("Mean Y w/INTENSIFY tactic"   = c(1,  1, rep(0, 6)),
               "Mean Y w/AUGMENT tactic"     = c(1, -1, rep(0, 6)),
               "Between groups difference"   = c(0,  2, rep(0, 6))))

                          Estimate 95% LCL 95% UCL     SE p-value
Mean Y w/INTENSIFY tactic   2.6181  2.3181  2.9182 0.1531  <1e-04
Mean Y w/AUGMENT tactic     3.2060  2.9028  3.5092 0.1547  <1e-04
Between groups difference  -0.5879 -1.0206 -0.1552 0.2208  0.0077

These results provide estimates for the mean outcome had everyone in the study treated according to each of these second-stage tactics. The difference in means (INTENSIFY - AUGMENT) is estimated to be **-0.5879** units, and the p-value for the test of the null hypothesis that this difference is zero is 0.0077. Thus, we have sufficient evidence to suggest that AUGMENT is a statistically-significantly better second-stage intervention tactic than INTENSIFY.

## <a name="part-c"></a>Part (c): Sample Size for Primary Aims involving Main Effect Comparisons

The question of whether there is a stage-1 main effect can be addressed with a t-test. Similarly, the second primary aim (whether there is a main effect of stage-2 treatment among non-responders) can be addressed with a t-test *among individuals who were re-randomized*. Therefore, we can use standard tools to compute sample size, e.g., http://statulator.com/SampleSize/ss2M.html or http://hedwig.mgh.harvard.edu/sample_size/js/js_parallel_quant.html, with slight modifications. 

We can also use R to do this for us. Let's first focus on the stage-1 main effect. Assuming we want to detect an effect size $\delta = 0.3$ with 85% power and a 5% significance level, the number of individuals required in each arm of the trial is 201, so the total sample size is 402.

In [17]:
power.t.test(delta = 0.9, power = 0.85, sig.level = 0.05, alternative = "two.sided")


     Two-sample t test power calculation 

              n = 23.171
          delta = 0.9
             sd = 1
      sig.level = 0.05
          power = 0.85
    alternative = two.sided

NOTE: n is number in *each* group


You can also compute this manually, if you're willing to use an asymptotic z-test. For large sample sizes, the results will be very similar; alternatively, you can make the assumption that your effect size is based on a known population standard deviation, and use the z-test. The formula is 
$$ n \geq \frac{4 (z_{1-\alpha/2} + z_{1-\beta})^2}{\delta^2}. $$

For the stage-2 main effect, we can use the above results, but we need to scale them appropriately, since the second-stage main effect only matters for non-responders in the ADHD design. So, to get the sample size for the full trial, we'll divide by the non-response rate:
$$ n \geq \frac{1}{1-r} \cdot \frac{4 (z_{1-\alpha/2} + z_{1-\beta})^2}{\delta^2} $$
where $r$ is the probability of response to first-stage treatment (assumed constant across first-stage treatments). As above, we can use `power.t.test()` and divide by $1-r$. Assuming a 50% response rate, the sample size for the whole trial is

In [18]:
r <- .5
ceiling(2 * power.t.test(delta = 0.3, power = 0.85, sig.level = 0.05)$n / (1-r))

[1] 802